# Item 41. Hashcode의 규약을 지켜라

## HashTable in java kotlin
- 해시테이블은 기본적으로 **배열**이다
- 최초배열 크기는 16이며 **임계점**은 75%로 12개의 요소가 들어오면 배열을 두배로 늘린다.
- 해시 셋, 맵은 내부에 해시 테이블을 사용하며 해시 테이블은 해시코드로된 키를 배열길이로 나눈 값으로 인덱스를 구하고 해당 인덱스의 값에 하나의 버킷에 값을 저장한다.
- 해시 배열을 늘릴떄 버킷의 인덱스 변경도 일어나며 이를 **리해싱**이라고 한다.
- 만약 동일한 해시코드를 가지고 있는 두 요소가 있다면 **해시충돌**이라고 표현하며, 버킷에 동일한 해시코드를 가진 모든 요소를 집어넣고, 조회시 이 요소들을 순회하면서 원하는 값을 찾는다.
- 이때 버킷은 연결 리스트로 구현되어 있다.
- 버킷내에 요소가 많아지면 성능이 떨어지기 때문에, 버킷의 크기를 늘리거나 해시 함수를 개선하는 방법으로 성능을 향상시킬 수 있다.
- 이때 버킷은 요소수가 8개 이상이면 트리로 변환된다.

## 해시충돌과 시간 복잡도, 성능

- 위에서 정리한 바와 같이 해시테이블은 기본적으로 index기반이므로 시간복잡도가 O(1)이다.
- 하지만 모든 요소가 해시 충돌이 일어나면 하나의 버킷에만 담기므로 성능이 O(n)으로 떨어진다.
- 물론 버킷사이즈가 일정크기이상이면 레드블랙트리로 전환되므로 O(logn)으로 성능을 유지할 수 있다.

## 실제 벤치마크 결과
```
private const val COUNT = 10000

@BenchmarkMode(Mode.AverageTime)
@OutputTimeUnit(TimeUnit.NANOSECONDS)
@State(Scope.Benchmark)
@Fork(value = 2, jvmArgs = ["-Xms4G", "-Xmx4G"]) // 4Gb의 힙 공간을 제공한 환경에서 두 번 벤치마크를 수행해 결과의 신뢰성 확보
open class HashTableVisitBenchmark {
     val crashMap: HashMap<HashEqualPerson, Int> = HashMap(COUNT)
     val nonCrashMap: HashMap<HashNotEqualPerson, Int> = HashMap(COUNT)
    init {
        for (i in 0 until COUNT) {
            crashMap[HashEqualPerson("name$i", 20)] = i
            nonCrashMap[HashNotEqualPerson("name$i", 20)] = i
        }
    }

    @Benchmark
    fun hashCrashTest(balckhole: Blackhole) {
        for (i in 0 until COUNT) {
            val found = crashMap[HashEqualPerson("name$i", 20)]
            balckhole.consume(found)
        }
    }

    @Benchmark
    fun hashNonCrashTest(balckhole: Blackhole) {
        for (i in 0 until COUNT) {
            val found = nonCrashMap[HashNotEqualPerson("name$i", 20)]
            balckhole.consume(found)
        }
    }
}
```

![uploaded image](https://github.com/jinia91/blogBackUp/blob/main/img/245526771773472?raw=true)

- 요소 1만개 기준 1만번 조회시 수천배이상 성능차이
- 단순시간복잡도보다 더 차이가나는 이유는뭘까? 레드블랙트리를 사용하니까 o(n)이 아니라 o(logn)이어야하는데?... 거의 o(n)정도로 성능이 떨어진다.

## 해시테이블과 가변성
- 해시테이블은 요소를 추가할때만 해시함수를 사용해 해시코드를 구하므로, 가변적인 객체를 키로 사용하면 문제가 발생할 수 있다.
- 정확히는 가변적인 상태를 기반으로 해시코드가 생성되는 경우, 해시코드가 변경되면 해시테이블에서 해당 객체를 찾을 수 없게 된다.

## 해시코드 규약
- 위에서처럼 해시코드를 기반으로한 해시테이블은 해시코드 변화에 민감하며, 동일 해시코드 생성에도 민감하다
- 따라서 해시코드를 구현할때는 다음 규약을 지켜야한다.
    1. 어떤 객체에 동등성이 유지된다면 해시코드도 유지되어야한다
    2. 어떤 객체가 equals 메서드에 의해 같다고 판단된다면, 두 객체의 해시코드는 같아야한다
    3. 추가적으로 해시코드는 최대한 많은 상태를 반영해 다르게 표현되는게 바람직하다

## Data class와 해시코드 / 동등성

- Data class는 equals, hashCode, toString을 자동으로 생성해준다.
- 하지만 data class는 가변성을 지원하므로(var), 불변객체가 아닐수 있으므로 해시코드가 변할 수 있어 주의해야한다!



true


Hello!
Hello!
